<a href="https://colab.research.google.com/github/ferdcoz/mcd-ast/blob/original/TP2/Codigo/TP2_Skforecast_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install skforecast session_info

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.8/424.8 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.1 MB/s eta 0:00:00
  Created wheel for session_info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=4ad44e3715a83921dcaa107686ddbc94c40e9bbe3857f2cd43517da84f302452
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session_info


## Librerias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

## Funciones

In [ ]:
def plot_data_split(train_data, val_data, test_data, column):
  fig = go.Figure()

  trace1 = go.Scatter(x=train_data.index, y=train_data[column], name="Training", mode="lines", line=dict(color="#440154"))
  trace2 = go.Scatter(x=val_data.index, y=val_data[column], name="Validation", mode="lines", line=dict(color="#3b528b"))
  trace3 = go.Scatter(x=test_data.index, y=test_data[column], name="Testing", mode="lines", line=dict(color="#21918c"))

  fig.add_trace(trace1)
  fig.add_trace(trace2)
  fig.add_trace(trace3)

  fig.update_layout(
      title=f"{column} - Data Split",
      xaxis_title="Fecha",
      yaxis_title="Cantidad",
      width  = 800,
      height = 400,
      margin=dict(l=20, r=20, t=35, b=20),
      legend=dict(
          orientation="h",
          yanchor="top",
          y=1.1,
          xanchor="left",
          x=0.001
      )
  )
  fig.show()

In [ ]:
def plot_prediction(train_data, val_data, test_data, predictions, column):
  fig = go.Figure()

  trace1 = go.Scatter(x=train_data.index, y=train_data[column], name="Training", mode="lines", line=dict(color="#440154"))
  trace2 = go.Scatter(x=val_data.index, y=val_data[column], name="Validation", mode="lines", line=dict(color="#3b528b"))
  trace3 = go.Scatter(x=test_data.index, y=test_data[column], name="Testing", mode="lines", line=dict(color="#21918c"))
  trace4 = go.Scatter(x=predictions.index, y=predictions['pred'], name="Prediction", mode="lines", line=dict(color="#fde725"))

  fig.add_trace(trace1)
  fig.add_trace(trace2)
  fig.add_trace(trace3)
  fig.add_trace(trace4)

  fig.update_layout(
      title=f"{column} predictions",
      xaxis_title="Fecha",
      yaxis_title="Cantidad",
      width  = 800,
      height = 400,
      margin=dict(l=20, r=20, t=35, b=20),
      legend=dict(
          orientation="h",
          yanchor="top",
          y=1.1,
          xanchor="left",
          x=0.001
      )
  )
  fig.show()

## Data

In [ ]:
# Incluyendo CC y la data como viene
url = 'https://raw.githubusercontent.com/ferdcoz/mcd-ast/original/TP2/Dataset/ST_ATENCION_ORIGNAL.csv'

# Excluyendo CC y sin 0s
#url = 'https://raw.githubusercontent.com/ferdcoz/mcd-ast/original/TP2/Dataset/ST_ATENCION.csv'
data = pd.read_csv(url, delimiter=';')
data.head()

,FECHA,CC,WAPP,TP,TAD,PP
0,2022-01-03,45,0,37,31,4
1,2022-01-04,63,0,24,24,5
2,2022-01-05,63,0,39,18,12
3,2022-01-06,71,0,33,26,5
4,2022-01-07,58,0,29,9,6


In [ ]:
data['FECHA'] = pd.to_datetime(data['FECHA'], format='%Y-%m-%d')
data = data.set_index('FECHA')
data = data.asfreq('B')
data = data.sort_index()
data.head()

,CC,WAPP,TP,TAD,PP
FECHA,,,,,
2022-01-03,45.0,0.0,37.0,31.0,4.0
2022-01-04,63.0,0.0,24.0,24.0,5.0
2022-01-05,63.0,0.0,39.0,18.0,12.0
2022-01-06,71.0,0.0,33.0,26.0,5.0
2022-01-07,58.0,0.0,29.0,9.0,6.0


In [ ]:
data = data.fillna(data.median())

In [ ]:
data_CC = data[['CC']]
data_WAPP = data[['WAPP']]
data_TP = data[['TP']]
data_TAD = data[['TAD']]
data_PP = data[['PP']]

## Modelo

In [ ]:
forecaster = ForecasterAutoreg(
                 regressor = XGBRegressor(random_state=123),
                 lags = 4
             )

## CC

In [ ]:
data = data_CC

### 1. Split data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "CC")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 24.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/12 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
  Backtesting metric: 84.59400934752937



In [ ]:
results_grid.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators
13,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",84.594009,0.01,3.0,500.0
18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",86.175659,0.10,3.0,100.0
15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",87.760276,0.01,5.0,500.0
19,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",88.962104,0.10,3.0,500.0
20,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",91.202139,0.10,5.0,100.0


### 4. Prediccion

In [ ]:
metric, predictions = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric:.2f}")
predictions.head()

  0%|          | 0/7 [00:00<?, ?it/s]

Backtest error: 155.33


,pred
2023-05-26,46.047474
2023-05-29,44.099991
2023-05-30,45.185471
2023-05-31,43.487389
2023-06-01,44.191547


### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions, "CC")

## WAPP

In [ ]:
data = data_WAPP

### 1. Split data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "WAPP")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 24.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/12 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500}
  Backtesting metric: 115.8580766914891



In [ ]:
results_grid.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators
17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",115.858077,0.01,10.0,500.0
16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",117.431942,0.01,10.0,100.0
22,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",117.878108,0.10,10.0,100.0
23,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",117.913237,0.10,10.0,500.0
8,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",120.390497,0.10,5.0,100.0


### 4. Prediccion

In [ ]:
metric, predictions = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric:.2f}")
predictions.head()

  0%|          | 0/7 [00:00<?, ?it/s]

Backtest error: 88.92


,pred
2023-05-26,25.889141
2023-05-29,29.577045
2023-05-30,26.469240
2023-05-31,21.191610
2023-06-01,26.380098


### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions, "WAPP")

## TP

In [ ]:
data = data_TP

### 1. Split data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "TP")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 24.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/12 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500}
  Backtesting metric: 885.1683979890361



In [ ]:
results_grid.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators
21,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",885.168398,0.10,5.0,500.0
19,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",901.519510,0.10,3.0,500.0
15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",977.828833,0.01,5.0,500.0
20,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",988.510037,0.10,5.0,100.0
18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",1022.790595,0.10,3.0,100.0


### 4. Prediccion

In [ ]:
metric, predictions = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric:.2f}")
predictions.head()

  0%|          | 0/7 [00:00<?, ?it/s]

Backtest error: 1132.65


,pred
2023-05-26,147.153763
2023-05-29,144.084244
2023-05-30,159.096191
2023-05-31,144.179520
2023-06-01,158.317886


### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions, "TP")

## TAD

In [ ]:
data = data_TAD

### 1. Split Data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "TAD")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 24.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/12 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
  Backtesting metric: 66.03090409354805



In [ ]:
results_grid.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators
12,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",66.030904,0.01,3.0,100.0
10,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",66.446898,0.10,10.0,100.0
11,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",66.848456,0.10,10.0,500.0
5,"[1, 2, 3, 4]","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",68.866398,0.01,10.0,500.0
4,"[1, 2, 3, 4]","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",69.574606,0.01,10.0,100.0


### 4. Prediccion

In [ ]:
metric, predictions = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric:.2f}")
predictions.head()

  0%|          | 0/7 [00:00<?, ?it/s]

Backtest error: 56.84


,pred
2023-05-26,34.722145
2023-05-29,36.158096
2023-05-30,34.606979
2023-05-31,35.579948
2023-06-01,34.495800


### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions, "TAD")

## PP

In [ ]:
data = data_PP

### 1. Split data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "PP")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 24.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/12 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500}
  Backtesting metric: 22.85589005433034



In [ ]:
results_grid.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators
5,"[1, 2, 3, 4]","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",22.855890,0.01,10.0,500.0
17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",29.078090,0.01,10.0,500.0
7,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",32.642184,0.10,3.0,500.0
8,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",33.777406,0.10,5.0,100.0
15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",34.195969,0.01,5.0,500.0


### 4. Prediccion

In [ ]:
metric, predictions = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric:.2f}")
predictions.head()

  0%|          | 0/7 [00:00<?, ?it/s]

Backtest error: 37.72


,pred
2023-05-26,17.168224
2023-05-29,17.389103
2023-05-30,16.030132
2023-05-31,14.807234
2023-06-01,14.089755


### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions, "PP")